In [1]:
from cffi import FFI
ffi = FFI()
with open('extracted_headers/safe_c_ffi_data.h','r') as f:
    myHeaders=f.read()
ffi.cdef(myHeaders)
with open('extracted_headers/safe_c_ffi_funcs.h','r') as f:
    funs=f.read()
ffi.cdef(funs)
lib=ffi.dlopen('../compiled_binaries/libsafe_app.so')

---

maybe it makes most sense to really define the data types by hand and just check them for changes with updates on the api - not entirely sure

- rid

---

```c
typedef unsigned long int uintptr_t;

typedef struct {
 char* id;
 char* scope;
 char* name;
 char* vendor;
} AppExchangeInfo;
```

In [2]:
class AppExchangeInfo:
    def __init__(self,id=b'noId',scope=b'noScope',name=b'noName',vendor=b'nobody'):
        self.id = ffi.new('char[]',id)
        self.scope = ffi.new('char[]',scope)
        self.name = ffi.new('char[]',name)
        self.vendor = ffi.new('char[]',vendor)

        self.entity = ffi.new('AppExchangeInfo *',[self.id,self.scope,self.name,self.vendor])

In [3]:
def AppExchangeInfo(id=b'noId',scope=b'noScope',name=b'noName',vendor=b'nobody'):
    id = ffi.new('char[]',id)
    scope = ffi.new('char[]',scope)
    name = ffi.new('char[]',name)
    vendor = ffi.new('char[]',vendor)

    myStruct = ffi.new('AppExchangeInfo *',[id,scope,name,vendor])
    
    return myStruct, [id, scope, name, vendor]

In [4]:
newExChangeInfo,infopayload = AppExchangeInfo()

In [5]:
ffi.string(newExChangeInfo.vendor)

b'nobody'

In [6]:
ffi.string(newExChangeInfo.name)

b'noName'

```c
typedef struct {
 _Bool read;
 _Bool insert;
 _Bool update;
 _Bool delete;
 _Bool manage_permissions;
} PermissionSet;
```

In [7]:
def PermissionSet(read=False,insert=False,update=False,delete=False,manage_permissions=False):
    return ffi.new('PermissionSet *',[read,insert,update,delete,manage_permissions])

In [8]:
newPermissionSet = PermissionSet()

```c
typedef struct {
 char* cont_name;
 PermissionSet access;
} ContainerPermissions;
```

In [9]:
def ContainerPermissions(name=b'noName',access=None):
    containerName = ffi.new('char[]',name)
    if not access:
        access = PermissionSet()
    container = ffi.new('ContainerPermissions *',[containerName,access[0]])
    
    return container, [containerName,access]

In [10]:
newPermissions,permission_addon=ContainerPermissions(b'myName2')

In [11]:
newPermissions.access.insert

False

```c
typedef struct {
 AppExchangeInfo app;
 _Bool app_container;
 ContainerPermissions* containers;
 uintptr_t containers_len;
 uintptr_t containers_cap;
} AuthReq;
```

In [12]:
def AuthReq(permissions,containers_len,containers_cap,id=b'noId',scope=b'noScope',name=b'noName',vendor=b'nobody',app_container=True):
    
    newExChangeInfo,infopayload = AppExchangeInfo(id,scope,name,vendor)
    
    authReq = ffi.new('AuthReq *',[newExChangeInfo[0],app_container,permissions,containers_len,containers_cap])
    
    return authReq, [newExChangeInfo,infopayload]

In [13]:
def AuthReq(permissions,containers_len,containers_cap,id=b'noId',scope=b'noScope',name=b'noName',vendor=b'nobody',app_container=True):
    
    newExChangeInfo,infopayload = AppExchangeInfo(id,scope,name,vendor)
    
    authReq = ffi.new('AuthReq *',[newExChangeInfo[0],app_container,permissions,containers_len,containers_cap])
    
    return authReq, [newExChangeInfo,infopayload]

In [14]:
myAuth,addData=AuthReq(newPermissions,1,1)

In [15]:
@ffi.callback("void(void *, FfiResult *)")
def feedback(userData, result):
    print(userData)
    print(result.error_code)

In [16]:
lib.app_set_additional_search_path(ffi.new('char[]',b'../compiled_binaries/'),ffi.NULL,feedback)

<cdata 'void *' NULL>
0


In [17]:
@ffi.callback("void(void *, FfiResult *, unsigned int, char *)")
def o_cb(userData, result, noClueWhatNumber, resultString):
    print(userData)
    print(result)
    print(noClueWhatNumber)
    print(resultString)

In [18]:
lib.encode_auth_req(myAuth,ffi.NULL,o_cb)

<cdata 'void *' NULL>
<cdata 'FfiResult *' 0x7ffe44f26958>
0
<cdata 'char *' NULL>


In [19]:
myAuth.containers.access.insert=True

In [20]:
myAuth.containers.access.delete=True

In [21]:
ffi.string(myAuth.app.name)

b'noName'

In [22]:
encodedAuth=None

In [23]:
@ffi.callback("void(void *, FfiResult *, unsigned int, char *)")
def o_cb(userData, result, reqId, encodedReq):
    global encodedAuth
    print(userData)
    print(result.error_code)
    print(reqId)
    print(ffi.string(encodedReq))
    encodedAuth = ffi.string(encodedReq)

In [47]:
lib.encode_auth_req(myAuth,ffi.NULL,o_cb)

<cdata 'void *' NULL>
0
3050472663
b'AAAAANeE0rUAAAAABAAAAAAAAABub0lkAQcAAAAAAAAAbm9TY29wZQYAAAAAAAAAbm9OYW1lBgAAAAAAAABub2JvZHkBAQAAAAAAAAAHAAAAAAAAAG15TmFtZTICAAAAAAAAAAEAAAADAAAA'


In [25]:
b'safe-auth://' + encodedAuth

b'safe-auth://AAAAAICGc1AAAAAABAAAAAAAAABub0lkAQcAAAAAAAAAbm9TY29wZQYAAAAAAAAAbm9OYW1lBgAAAAAAAABub2JvZHkBAQAAAAAAAAAHAAAAAAAAAG15TmFtZTICAAAAAAAAAAEAAAADAAAA'

In [26]:

sysUri = FFI()
with open('extracted_headers/safe_c_ffi_data.h','r') as f:
    myHeaders=f.read()
sysUri.cdef(myHeaders)
with open('extracted_headers/system_uri.h','r') as f:
    funs=f.read()
sysUri.cdef(funs)
lib2=sysUri.dlopen('../compiled_binaries/libsystem_uri.so')

In [27]:
@sysUri.callback("void(void *, FfiResult *)")
def installerFeedback(userData, result):
    print(userData)
    print(result.error_code)

```c
void install(char* bundle, 
             char* vendor, 
             char* name, 
             char* exec_args, 
             uint_least64_t exec_args_len, 
             char* icon, 
             char* schemes, 
             void* user_data, 
             void (*o_cb)(void* user_data, FfiResult* result));
```

In [28]:
print(ffi.string(myAuth.app.name))
print(ffi.string(myAuth.app.vendor))
print(ffi.string(myAuth.app.scope))

b'noName'
b'nobody'
b'noScope'


In [29]:
bundle=sysUri.new('char[]',b'nobundle')
vendor=sysUri.new('char[]',b'nobody')
name=sysUri.new('char[]',b'noName')
exec_args=sysUri.new('char[]',b'')#sysUri.NULL
exec_args_len=0
icon=sysUri.new('char[]',b'')#sysUri.NULL
schemes=sysUri.new('char[]',b'')#sysUri.NULL
user_data=sysUri.NULL

In [30]:
lib2.install(bundle,vendor,name,exec_args,exec_args_len,icon,schemes,user_data,installerFeedback)

<cdata 'void *' NULL>
0


```c 
void open_uri(char* uri, 
              void* user_data, 
              void (*o_cb)(void* user_data, FfiResult* result));
```

In [31]:
@sysUri.callback("void(void *, FfiResult *)")
def openerFeedback(userData, result):
    global res
    res = result
    print(userData)
    print(result.error_code)

In [32]:
uri=sysUri.new('char[]',b'safe-auth://' + encodedAuth)
user_data=sysUri.NULL

In [46]:
lib2.open_uri(uri,user_data,openerFeedback)

<cdata 'void *' NULL>
0


```c 
void decode_ipc_msg(char* msg, 
                    void* user_data, 
                    void (*o_auth)(void* user_data, uint32_t req_id, AuthGranted* auth_granted), 
                    void (*o_unregistered)(void* user_data, uint32_t req_id, uint8_t* serialised_cfg, 
                                            uintptr_t serialised_cfg_len), 
                    void (*o_containers)(void* user_data, uint32_t req_id), 
                    void (*o_share_mdata)(void* user_data, uint32_t req_id), 
                    void (*o_revoked)(void* user_data), 
                    void (*o_err)(void* user_data, FfiResult* result, uint32_t req_id));
```

In [34]:
@ffi.callback("void(void *, uint32_t, AuthGranted *)")
def o_auth_decode_ipc_msg(userData, req_id, auth_granted):
    global grantedAuth
    grantedAuth = auth_granted
    print('o_auth')
    print(userData)
    print(reqId)

In [35]:
@ffi.callback("void(void *, uint32_t, uint8_t *, uintptr_t)")
def o_unregistered_decode_ipc_msg(userData, req_id, serialised_cfg, serialised_cfg_len):
    print('o_unregistered')
    print(reqId)

In [36]:
@ffi.callback("void(void *, uint32_t)")
def o_containers_decode_ipc_msg(userData, req_id):
    print('o_containers')
    print(reqId)

In [37]:
@ffi.callback("void(void *, uint32_t)")
def o_share_mdata_decode_ipc_msg(userData, req_id):
    print('o_share')
    print(reqId)

In [38]:
@ffi.callback("void(void *)")
def o_revoked_decode_ipc_msg(userData):
    print('o_revoked')
    print(reqId)

In [39]:
@ffi.callback("void(void *, FfiResult *, uint32_t)")
def o_error_decode_ipc_msg(userData, result, req_id):
    global res
    res = result
    print('o_err')
    print(userData)
    print(req_id)
    print(result.error_code)
    print(ffi.string(result.description))

In [48]:
msg = ffi.new('char[]',b'safe-auth://' + encodedAuth)

In [49]:
lib.decode_ipc_msg(msg,ffi.NULL,o_auth_decode_ipc_msg,o_unregistered_decode_ipc_msg
                   ,o_containers_decode_ipc_msg,o_share_mdata_decode_ipc_msg,
                   o_revoked_decode_ipc_msg,o_error_decode_ipc_msg)

o_err
<cdata 'void *' NULL>
0
-1
b'Serialisation error'


In [42]:
msg = ffi.new('char[]',encodedAuth)

In [43]:
lib.decode_ipc_msg(msg,ffi.NULL,o_auth_decode_ipc_msg,o_unregistered_decode_ipc_msg
                   ,o_containers_decode_ipc_msg,o_share_mdata_decode_ipc_msg,
                   o_revoked_decode_ipc_msg,o_error_decode_ipc_msg)

o_err
<cdata 'void *' NULL>
0
-202
b'IPC error: InvalidMsg'


In [37]:
uriAddress=ffi.new('char[]',b'safe-auth://' + encodedAuth)

In [ ]:
uriAddress=ffi.new('char[]',b'safe-auth://' + encodedAuth)

In [ ]:
void open_uri(char* uri, void* user_data, void (*o_cb)(void* user_data, FfiResult* result));

In [ ]:
uriAddress=ffi.new('char[]',b'safe-auth://' + encodedAuth)

In [ ]:
lib2.open(uriAddress)